# Delta Project Controller

### Packages

In [2]:
from stupidArtnet import StupidArtnet
import sys
import time
import os
import PySimpleGUI as sg
import pygame_gui
import pygame
from pygame.event import Event
from pygame_gui.elements import UIButton
from pygame_gui.windows import UIColourPickerDialog
from pynput.mouse import Listener
import pyautogui
from PIL import Image, ImageGrab

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


### Constants

In [3]:
# Setup Constants
DEFAULT_PACKET_SIZE = 512
DEFAULT_FPS = 30
ENFORCE_EVEN_PACKET = True
ENFORCE_BROADCAST = True
DEFAULT_UNIVERSE_ID = 1
DEFAULT_CHANNEL_START_ID = 1
DEFAULT_CHANNEL_WIDTH = 11
# Control Constants
DIMMER_ID = 1
DIMMER_FINE_ID = 2
STROBE_ID = 3
RED_ID = 4
GREEN_ID = 5
BLUE_ID = 6
WHITE_ID = 7
AMBER_ID = 8
UV_ID = 9
PRESET_ID = 10
SOUND_ID = 11
DEFAULT_LIGHT_VALUE = [255]+[0]*(DEFAULT_CHANNEL_WIDTH-1)
RESET_VALUE = [0]*DEFAULT_CHANNEL_WIDTH
LIGHT_OFF_VALUE = RESET_VALUE
# Mappings
FIXTURE_TO_ID_DICT = {'Dimmer':DIMMER_ID,'Dimmer_Fine':DIMMER_FINE_ID,
                    'Strobe':STROBE_ID,'Red':RED_ID,'Green':GREEN_ID,
                    'Blue':BLUE_ID,'White':WHITE_ID,'Amber':AMBER_ID,
                    'UV':UV_ID,'Preset':PRESET_ID,'Sound':SOUND_ID}
ID_TO_FIXTURE_DICT = dict([(i,fixture) for fixture,i in FIXTURE_TO_ID_DICT.items()])
DEFAULT_GROUPS = {'main_group':['light_1','light_2','light_3'],
                 'ambiance_group':['light_4','light_5','light_6']}
# Events
EVENTS_SET = {'group_1','group_2','light_1','light_2','light_3',
              'light_4','light_5','light_6'}

### Code

#### Objects

##### Channel

In [4]:
class Channel:
    """
    """
    def __init__(self,server:StupidArtnet,channel_start:int,channel_width:int):
        """
        """
        self.server = server
        self.channel_start = channel_start
        self.channel_width = channel_width
        self.offset = self.channel_start - 1

    def set_value(self,fixture_id:int,value:int,show=True):
        """
        """
        if value < 0 or value > 255:
            raise ValueError(f'The value for {ID_TO_FIXTURE_DICT[fixture_id]} should be contained in [0,255]')
        self.server.set_single_value(self.offset+fixture_id,value)
        time.sleep(0.002)
        if show:
            self.server.show()
        time.sleep(0.002)

    def set_values(self,values:list):
        """
        """
        if len(values) != self.channel_width:
            raise ValueError(f'The list of values sent by the channel must be of size equal to the channel width: {self.channel_width}')
        for fixture_id, value in enumerate(values):
            self.set_value(fixture_id+1, value, show=False)

    def reset(self):
        """ """
        self.set_values(RESET_VALUE)

##### Light Source

In [5]:
class LightSource:
    """ Abstract Light Source Object """
    
    def __init__(self,name:str,state=DEFAULT_LIGHT_VALUE.copy()):
        """ Instantiate the Light Source, defined by a name and a state. """
        self.name = name
        self.state = state

    def set_fixture_value(self,fixture_id:int, value:int):
        """ Define the value of a fixture. """

    def set_fixture_values(self,fixture_id:int, value:int):
        """ Define the values of all fixtures. """

    def blink():
        """ Make the Light Source Blink. """

    def set_rgb(self, values:list):
        """ Define the RGB values. """

    def turn_off(self):
        """ Turn off the Light Source"""

    def turn_on(self):
        """ Turn on the Light Source"""

    def reset(self):
        """ Reset the Light Source to its default state. """

###### Light

In [6]:
class Light(LightSource):
    """
    Light object class. A light is model by the channel to which
    it is linked and a state. The latter is a list containing the 
    values for each of the fixture in the channel. 
    
    """
    
    def __init__(self,name:str, channel:Channel):
        """
        Create a Light instance.

        :param name: String identifier for the light. If the light is used in a Group,
                     please be sure to enter unique identifiers.
        :param channel: ArtNet channel object to which the light is bound. 

        """
        super().__init__(name)
        self.group_name = ''
        self.channel = channel
        self.turn_on()
        
    def set_fixture_value(self,fixture_id:int,value:int):
        """
        Set the fixture to the given value.

        :param fixture_id: Id of the fixture to be set.
        :param value: Integer value of the fixture, should be contained in [0,255].
        
        """
        new_state = self.state.copy()
        new_state[fixture_id-1] = value
        self.channel.set_value(fixture_id, value)
        self.state = new_state

    def set_fixture_values(self,values=[]):
        """
        Set the fixtures to the given list of values.

        :param values: Ordered list of values to be set.
        
        """
        self.channel.set_values(values)
        new_state = values
        self.state = new_state

    def set_rgb(self, values:list):
        """
        Set the RGB fixtures to the color code given in values.

        :param values: List containing the values for the red, green and blue fixtures.
        
        """
        for idx, fixture_id in enumerate([RED_ID,GREEN_ID,BLUE_ID]):
            self.set_fixture_value(fixture_id,values[idx])

    def blink(self,blink_time=0.2,n_repeat=2):
        """ """
        prev_state = self.state.copy()
        self.turn_off()
        time.sleep(blink_time)
        self.reset()
        self.set_fixture_value(WHITE_ID, 255)
        for i in range(n_repeat):
            time.sleep(blink_time)
            self.turn_on()
            time.sleep(blink_time)
            self.turn_off()
        self.set_fixture_values(prev_state)
        self.turn_on()

    def turn_off(self):
        """ Turn off the light by setting dimmer to 0. """
        self.set_fixture_value(DIMMER_ID,0)

    def turn_on(self):
        """ Turn on the light by setting dimmer to 255. """
        self.set_fixture_value(DIMMER_ID,255)

    def reset(self):
        """ Reset the light to its default state, i.e. zero value for each fixture. """
        self.turn_off()
        self.channel.reset()
        new_state = RESET_VALUE
        self.state = new_state

###### Group

In [7]:
class Group(LightSource):
    """
    Group object class. A group is modeled by a list
    of lights. Every action applied to the group
    will be executed on all lights present. Each light in the 
    group should be unique and have a unique name.
    
    """
    
    def __init__(self,name:str, lights=[]):
        """
        Create a Group instance.

        :param name: String identifier for the group of lights.
        :param lights: List containing the initial lights of the group,
                       empty by default. All lights should be unique.
        
        """
        super().__init__(name)
        self.lights = []
        self.light_names = set()
        if len(self.light_names) != len(self.lights):
            raise ValueError('Duplicate names in the list of lights provided to the Group constructor')
        for l in lights:
            self.add_light(l)

    def add_light(self,light:Light):
        """
        Add a light to the existing pool. Must be a new unique light.

        :param light. Light object to add to the existing pool of lights in the group.
        
        """
        if light.name in self.light_names:
            raise ValueError('Tried to add a light whose name is already present in the group.')
        if light.group_name != '' and light.group_name != self.name:
            raise ValueError('Tried to add a light which is already present in another group.') 
        self.lights.append(light)
        light.set_fixture_values(self.state)
        light.group_name = self.name
        self.light_names = self.light_names.union(set(light.name))

    def remove_light(self,light_name:str):
        """
        Remove a light from the existing pool.

        :param light_name: Identifier of the light to be removed.
        
        """
        light.group_name = ''
        self.light_names = self.light_names.difference()
        self.lights = [l for l in self.lights if l.name != light_name]
        self.light.reset()
        
        
    def set_fixture_value(self, fixture_id:int, value:int):
        """
        Set the given fixture to 'value' for all lights in the group.

        :param fixture_id: Id of the fixture to be set.
        :param value: Integer value of the fixture, should be contained in [0,255].
        
        """
        for l in self.lights:
            l.set_fixture_value(fixture_id,value)

    def set_rgb(self, values:list):
        """
        Set the RGB fixtures to the color code given in values.

        :param values: List containing the values for the red, green and blue fixtures.
        
        """
        for idx, fixture_id in enumerate([RED_ID,GREEN_ID,BLUE_ID]):
            self.set_fixture_value(fixture_id,values[idx])

    def blink(self):
        """ Make all lights in the group blink. """
        for l in self.lights:
            l.blink()

    def turn_off(self):
        """ Turn off the light by setting dimmer to 0. """
        self.set_fixture_value(DIMMER_ID,0)

    def turn_on(self):
        """ Turn on the light by setting dimmer to 255. """
        self.set_fixture_value(DIMMER_ID,255)

    def reset(self):
        """ Reset all lights in the group to their default states, i.e. zero value for each fixture. """
        for l in self.lights:
            l.reset()

#### Color Picker

In [8]:
#pygame.Rect(160, 50, 420, 400)

In [9]:
def color_popup(light_object:LightSource):
    """
    Popup window to select the color of the given light object.

    :param light_object: LightSource which will be controlled.
    
    """
    SCREEN = pygame.display.set_mode((400, 400))
    SCREEN.fill(pygame.Color(0, 0, 0))
    pygame.display.init()
    pygame.display.update()
    pygame.init()
    pygame.display.set_caption('Colour Picking App')

    
    ui_manager = pygame_gui.UIManager((400, 400))
    colour_picker = None                                    
    current_colour = pygame.Color(0, 0, 0)
    
    clock = pygame.time.Clock()
    begin_loop = True
    
    while True:
        time_delta = clock.tick(60) / 1000
        event_list = pygame.event.get()
        if begin_loop == True:
            event_list.append(Event(1,{'state':1,'gain':1}))
        for event in event_list:
            if begin_loop == True:
                colour_picker = UIColourPickerDialog(pygame.Rect(0, 0, 400, 400),
                                                    ui_manager,
                                                    window_title=f"Change Colour of {light_object.name}",
                                                    initial_colour=current_colour,
                                                    light_object=light_object)
                begin_loop = False
            if (event.type == pygame.QUIT or event.type == pygame_gui.UI_COLOUR_PICKER_COLOUR_PICKED
                or (event.type == 1 and event.gain == 0)):
                pygame.quit()
                time.sleep(1)
                return
            ui_manager.process_events(event)
            
        ui_manager.update(time_delta)
        ui_manager.draw_ui(SCREEN)
        pygame.display.update()
    
    pygame.quit()

#### GUI

In [10]:
def create_UI_layout() -> list:
    """ Create the user interface layout using PySimpleGui. """
    layout = [[sg.Text("Control Center Group 1",justification="center",s=(47,1))],
    [sg.Button("Main",button_color="black on SkyBlue1",key="group_1",s=(10,5)),
        sg.Button("Light 1",button_color="black on SkyBlue1",key="light_1",s=(10,5)),
        sg.Button("Light 2",button_color="black on SkyBlue1",key="light_2",s=(10,5)),
        sg.Button("Light 3",button_color="black on SkyBlue1",key="light_3",s=(10,5))],
    [sg.Text("Control Center Group 2",justification="center",s=(47,1))],
    [sg.Button("Ambiance",button_color="black on gold",key="group_2",s=(10,5)),
     sg.Button("Light 4",button_color="black on gold",key="light_4",s=(10,5)),
     sg.Button("Light 5",button_color="black on gold",key="light_5",s=(10,5)),
     sg.Button("Light 6",button_color="black on RoyalBlue1",key="light_6",s=(10,5))],
    [sg.Text("Mouse Coord:"), sg.Text(size=20, key='Coordinate')],
    [sg.Image('color_wheel.png',size=(512,512))],
    [sg.Graph((450, 450), (0, 0), (450, 450),background_color='lightblue', enable_events=True, key='-GRAPH-')]]
    return layout

In [14]:
def UI_process(light_object_dict:dict):
    """ User interface process, handling user actions. 
    
    :param light_object_dict: Dictionnary with event_id as key and 
                              corresponding light object as value.
    
    """
    layout = create_UI_layout()
    window = sg.Window("Delta Control", layout, background_color='black', resizable=True).finalize()
    window.bind('<Motion>', 'Motion')
    #
    graph = window['-GRAPH-']
    embed = graph.TKCanvas
    os.environ['SDL_WINDOWID'] = str(embed.winfo_id())
    os.environ['SDL_VIDEODRIVER'] = 'windib'
    #
    def on_move(x, y):
        window.write_event_value('Motion', (x, y))
    #
    position = pyautogui.position()
    #listener = Listener(on_move=on_move)
    #listener.start()
    #
    while True:
        # Update GUI
        event, values = window.read(timeout=1000)
        if event == sg.WIN_CLOSED:
            for light_object in light_object_dict.values():
                light_object.turn_off()
            #listener.stop()
            break
        elif event == 'Motion':
            
            e = window.user_bind_event
            window['Coordinate'].update(f'({e.x_root}, {e.y_root})')
            pixel = ImageGrab.grab(bbox =(e.x_root, e.y_root, e.x_root+1, e.y_root+1)).getdata()[0]
            """
            x,y = values[event]
            print(x,y)
            pixel = ImageGrab.grab(bbox =(x, y, x+1, y+1)).getdata()[0]
            print(pixel)
            """
            hex_color = sg.rgb(*pixel)
            window['Coordinate'].update(text_color=hex_color)
        elif event in EVENTS_SET:
            light_object = light_object_dict[event]
            color_popup(light_object)
    window.close();

#### Pipeline

In [12]:
def live_color_picker(ip:str, num_lights:int, groups_mapping=DEFAULT_GROUPS,
                      packet_size=DEFAULT_PACKET_SIZE, fps=DEFAULT_FPS,
                      even_packet_size = ENFORCE_EVEN_PACKET, broadcast=ENFORCE_BROADCAST,
                      universe_id=DEFAULT_UNIVERSE_ID,channel_width=DEFAULT_CHANNEL_WIDTH):
    """
    Pipeline to select color of each light source in real time.

    :param ip: Ip of the ArtNet receiving device.
    :param num_lights: Number of lights to be configured. 
    :param groups_mapping: Mapping between group name and set of lights.
    :param packet_size: Size of ArtNet packets.
    :param fps: Refresh rate of the server.
    :param even_packet_size: Boolean variable to enforce even packets (May be
                             required by the receiver).
    :param broadcast: Boolean variable to allow broadcast in the subnet.
    :param universe_id: Identifier of the universe with which we want to communicate.
    :param channel_width: Number of fixtures per channel.
    
    """
    # Init connections
    server = StupidArtnet(ip,universe_id,packet_size,fps,even_packet_size,broadcast)
   # Lights
    lights = []
    for i in range(num_lights):
        channel_start = DEFAULT_CHANNEL_START_ID + i*channel_width
        lights.append(Light(name='light_'+str(i+1),channel=Channel(server,channel_start,channel_width)))
    # Groups
    groups = []
    for group_name, group_lights_names in groups_mapping.items():
        group_lights = [l for l in lights if l.name in group_lights_names]
        groups.append(Group(name=group_name, lights=group_lights))
    # light Object Mapping
    light_object_dict = [('light_'+str(i+1),lights[i]) for i in range(num_lights)]
    light_object_dict.extend([('group_'+str(i+1),groups[i]) for i in range(len(groups))])
    light_object_dict = dict(light_object_dict)
    # UI Loop
    UI_process(light_object_dict)

#### Tests

In [15]:
live_color_picker('169.254.79.148',6)

In [32]:
# using the grab method
im2 = ImageGrab.grab(bbox =(0,0,1920,1080))
    
im2.show()

Init signature:
sg.Window(
    title,
    layout=None,
    default_element_size=None,
    default_button_element_size=(None, None),
    auto_size_text=None,
    auto_size_buttons=None,
    location=(None, None),
    relative_location=(None, None),
    size=(None, None),
    element_padding=None,
    margins=(None, None),
    button_color=None,
    font=None,
    progress_bar_color=(None, None),
    background_color=None,
    border_depth=None,
    auto_close=False,
    auto_close_duration=3,
    icon=None,
    force_toplevel=False,
    alpha_channel=None,
    return_keyboard_events=False,
    use_default_focus=True,
    text_justification=None,
    no_titlebar=False,
    grab_anywhere=False,
    grab_anywhere_using_control=True,
    keep_on_top=None,
    resizable=False,
    disable_close=False,
    disable_minimize=False,
    right_click_menu=None,
    transparent_color=None,
    debugger_enabled=True,
    right_click_menu_background_color=None,
    right_click_menu_text_color=None,
 

---

In [11]:
server = StupidArtnet('169.254.79.148', 1, 512, 30, True, True)

In [12]:
c = Channel(server,23,11)

In [13]:
l = Light('test',c)

[255, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [14]:
l.set_rgb([255,0,255])

In [18]:
l.turn_on()

In [19]:
l.turn_off()

In [18]:
l.blink(n_repeat=2)

#### Archive

In [ ]:
def color_popup(light_object:LightSource):
    """
    Popup window to select the color of the given light object.

    :param light_object: LightSource which will be controlled.
    
    """
    SCREEN = pygame.display.set_mode((400, 400))
    SCREEN.fill(pygame.Color(0, 0, 0))
    pygame.display.init()
    pygame.display.update()
    pygame.init()

    pygame.display.set_caption('Colour Picking App')
    
    
    ui_manager = pygame_gui.UIManager((400, 400))
    #background = pygame.Surface((400, 400))
    #background.fill("#3a3b3c")
    """
    colour_picker_button = UIButton(relative_rect=pygame.Rect(-180, -60, 150, 30),
                                    text='Pick Colour',
                                    manager=ui_manager,
                                    anchors={'left': 'right',
                                            'right': 'right',
                                            'top': 'bottom',
                                            'bottom': 'bottom'})

    """
    colour_picker = None                                    
    current_colour = pygame.Color(0, 0, 0)
    #picked_colour_surface = pygame.Surface((400, 400))
    #picked_colour_surface.fill(current_colour)
    
    clock = pygame.time.Clock()
    begin_loop = True

    while True:
        time_delta = clock.tick(60) / 1000
        event_list = pygame.event.get()
        if begin_loop == True:
            event_list.append(Event(1,{'state':1,'gain':1}))
        for event in event_list:
            if begin_loop == True:
                colour_picker = UIColourPickerDialog(pygame.Rect(0, 0, 400, 400),
                                                    ui_manager,
                                                    window_title="Change Colour...",
                                                    initial_colour=current_colour,
                                                    light_object=light_object)
                #colour_picker_button.disable()
                begin_loop = False
            if event.type == pygame.QUIT:
                pygame.quit()
                break
            """
            if event.type == pygame_gui.UI_BUTTON_PRESSED and event.ui_element == colour_picker_button:
                colour_picker = UIColourPickerDialog(pygame.Rect(0, 0, 400, 400),
                                                    ui_manager,
                                                    window_title="Change Colour...",
                                                    initial_colour=current_colour,
                                                    light_object=light_object)
                colour_picker_button.disable()
            """
            if event.type == pygame_gui.UI_COLOUR_PICKER_COLOUR_PICKED:
                pygame.quit()
                return
                #current_colour = event.colour
                #picked_colour_surface.fill(current_colour)
            if event.type == 1:
                if event.gain == 0:
                    pygame.quit()
                    return
            if event.type == pygame_gui.UI_WINDOW_CLOSE:
                #colour_picker_button.enable()
                colour_picker = None
            
            ui_manager.process_events(event)
            
        ui_manager.update(time_delta)
    
        #SCREEN.blit(background, (0, 0))
        #SCREEN.blit(picked_colour_surface, (200, 100))
    
        ui_manager.draw_ui(SCREEN)
    
        pygame.display.update()
    
    pygame.quit()